In [ ]:
import requests
from dotenv import load_dotenv
import os
import json
import base64   
import pandas as pd
import librosa
import numpy as np
import lyricsgenius
import langdetect
import re
import string
import tempfile
from datetime import datetime, timedelta


In [ ]:
# For any api we can try using the "+" email trick to get more API keys

genius_client_id = "wZZ2RWc5mqp-5Pbz2W1rQJWE8LQ3pFBrb1Hw5_AOqgybq28mt7kjdjcG4zktCNbO"
genius_client_secret = "PefqBJHor_muDgTutGlaXXaxmzsI7TQCps9FQ3FwkUTT0WJIT3s0A5YA9mnFbfp_-CBhQF7b0omgE8kaM3dJ3w"
genius_access_token = "NUHHVpwnmbDYUYw8Padu0gQeHvYN4OsKYtE2MKNUpBUI6yR-xZXKY6S5NvCnFbiP"

lastfm_api_key = "97d5a64d5ba4a8bc580b752ceff3b87f"
lastfm_secret = "35175090bd61f6f16ac607bd26e5b1de"

In [ ]:
base_url = 'http://ws.audioscrobbler.com/2.0/'

def lastfm_get(payload):
    headers = {'user-agent': 'DataCollectorBot'}
    payload['api_key'] = lastfm_api_key
    payload['format'] = 'json'
    response = requests.get(base_url, headers=headers, params=payload)
    return response.json()

# Function for each endpoint
def get_user_info(user):
    payload = {'method': 'user.getinfo', 'user': user}
    return lastfm_get(payload)

def get_loved_tracks(user):
    payload = {'method': 'user.getlovedtracks', 'user': user}
    return lastfm_get(payload)

def get_top_albums(user):
    payload = {'method': 'user.gettopalbums', 'user': user}
    return lastfm_get(payload)

def get_top_artists(user):
    payload = {'method': 'user.gettopartists', 'user': user}
    return lastfm_get(payload)

def get_top_tags(user):
    payload = {'method': 'user.gettoptags', 'user': user}
    return lastfm_get(payload)

def get_top_tracks(user):
    payload = {'method': 'user.gettoptracks', 'user': user}
    return lastfm_get(payload)



In [ ]:
def list_to_df(data_list, columns):
    if data_list:
        df = pd.DataFrame(data_list, columns=columns)
        return df
    else:
        return pd.DataFrame(columns=columns)
    
def get_top_albums_df(user):
    result = get_top_albums(user)
    albums = []
    for item in result['topalbums']['album']:
        albums.append({
            'Rank': item['@attr']['rank'],
            'Album Name': item['name'],
            'Artist': item['artist']['name'],
            'Play Count': item['playcount']
        })
    return list_to_df(albums, ['Rank', 'Album Name', 'Artist', 'Play Count'])

def get_top_tags_df(user):
    result = get_top_tags(user)  # Fetch the data from the API
    tags = []
    # Check if the 'tag' data is available in the result
    if 'toptags' in result and 'tag' in result['toptags']:
        for item in result['toptags']['tag']:
            tags.append({
                'Tag Name': item['name'],
                'Count': item['count']
            })
    return pd.DataFrame(tags, columns=['Tag Name', 'Count'])


def get_top_artists_df(user):
    result = get_top_artists(user)
    artists = []
    for item in result['topartists']['artist']:
        artists.append({
            'Rank': item['@attr']['rank'],
            'Artist': item['name'],
            'Play Count': item['playcount']
        })
    return list_to_df(artists, ['Rank', 'Artist', 'Play Count'])


def get_top_tracks_df(user):
    result = get_top_tracks(user)
    tracks = []
    for item in result['toptracks']['track']:
        tracks.append({
            'Rank': item['@attr']['rank'],
            'Track Name': item['name'],
            'Artist': item['artist']['name'],
            'Play Count': item['playcount']
        })
    return list_to_df(tracks, ['Rank', 'Track Name', 'Artist', 'Play Count'])